# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [121]:
import pandas as pd
import requests

In [122]:
# 1) Linhas duplicadas

sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape

# Não há duplicados

(27028, 69)


(27028, 69)

In [123]:
# 2) Valores Missing por variável
sinasc.isna().sum()


ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [124]:
# 3) Valores missing de algumas variáveis:
# ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 

sinasc_select = sinasc[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]
sinasc_select
sinasc_select.isnull().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [125]:
# 4) Deletando os dados missing de Apgar5
print(sinasc_select['APGAR5'].isnull().sum())

sinasc_apgar5 = sinasc_select[~sinasc_select['APGAR5'].isna()]

print(sinasc_apgar5.shape)


103
(26925, 9)


In [128]:
sinasc_apgar5.sample(20)

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
1933,1,32,2.0,12 anos ou mais,0.0,37 a 41 semanas,Única,4,9.0
5281,1,16,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
6985,1,17,1.0,1 a 3 anos,NaN,37 a 41 semanas,Única,3,9.0
4506,1,19,1.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
6213,1,28,1.0,12 anos ou mais,0.0,32 a 36 semanas,Única,3,9.0
12951,1,18,2.0,4 a 7 anos,NaN,32 a 36 semanas,Única,4,10.0
22919,1,33,2.0,4 a 7 anos,5.0,37 a 41 semanas,Única,4,9.0
10698,1,29,2.0,12 anos ou mais,1.0,37 a 41 semanas,Única,4,9.0
25213,1,22,2.0,4 a 7 anos,1.0,37 a 41 semanas,Única,4,9.0
17725,1,25,1.0,4 a 7 anos,0.0,37 a 41 semanas,Única,4,8.0


In [141]:
sinasc_apgar5['CONSULTAS'].isna().sum()

0

In [143]:
sinasc_tratado = sinasc_apgar5.copy() #criando uma cópia do df original para fazer as alterações

In [144]:
# 5) ignorado 

tratado = sinasc_apgar5['ESTCIVMAE'].fillna(9) # retorna apenas a coluna alterada

sinasc_tratado['ESTCIVMAE'] = tratado # substituindo a coluna original pela tratada

# A coluna 'CONSULTAS' não possui missings

In [152]:
# 6) Substituindo valores por zero

sinasc_tratado['QTDFILVIVO'].fillna(0, inplace=True)

sinasc_tratado

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [153]:
# 7) Substituições

# Usarei a mediana para os valores numéricos, e 'não preenchido' para os categóricos. 
#Escolhi a mediana ao invés da média por serem numeros inteiros

# Os itens 'escolaridade da mãe' e 'local de nascimentos' não tem dados faltantes


# Não preenchido: Escolaridade mãe, Gestação, Gravidez

sinasc_tratado['ESCMAE'].fillna('Não preenchido', inplace=True)
sinasc_tratado['GESTACAO'].fillna('Não preenchido', inplace=True)
sinasc_tratado['GRAVIDEZ'].fillna('Não preenchido', inplace=True)

sinasc_tratado.sample(30)


,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
825,1,33,2.0,8 a 11 anos,3.0,37 a 41 semanas,Não preenchido,4,9.0
9449,1,18,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
383,1,30,5.0,8 a 11 anos,1.0,37 a 41 semanas,Única,3,10.0
12048,1,18,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,3,9.0
23686,1,34,5.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
26001,1,17,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
17929,1,32,2.0,12 anos ou mais,3.0,Não preenchido,Única,4,10.0
2665,1,25,1.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,9.0
5458,1,33,2.0,12 anos ou mais,1.0,37 a 41 semanas,Única,4,9.0
13297,1,22,5.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0


In [154]:
# 8) Classificação e frequência


sinasc_tratado.loc[sinasc_tratado['APGAR5'] <= 3, 'Classificação'] = 'asfixia severa'
sinasc_tratado.loc[(sinasc_tratado['APGAR5'] > 3) & (sinasc_apgar5['APGAR5'] <= 5), 'Classificação'] = 'asfixia moderada'
sinasc_tratado.loc[(sinasc_tratado['APGAR5'] > 5) & (sinasc_apgar5['APGAR5'] <= 7), 'Classificação'] = 'asfixia leve'
sinasc_tratado.loc[sinasc_tratado['APGAR5'] > 7, 'Classificação'] = 'normal'

sinasc_tratado.head(30)

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,Classificação
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
5,1,33,5.0,1 a 3 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
6,1,37,1.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
7,1,27,2.0,12 anos ou mais,0.0,37 a 41 semanas,Única,4,10.0,normal
8,1,20,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,3,9.0,normal
9,1,30,2.0,1 a 3 anos,1.0,37 a 41 semanas,Única,4,10.0,normal


In [155]:
# Frequência

contagem = sinasc_tratado['Classificação'].value_counts()
 # 26925

frequencia = (contagem / contagem.sum()) * 100
frequencia


normal              98.284123
asfixia leve         1.188487
asfixia severa       0.274838
asfixia moderada     0.252553
Name: Classificação, dtype: float64

In [156]:
# 9) Renomeando as colunas

colunas = list(sinasc_tratado.columns.values)
colunas


['LOCNASC',
 'IDADEMAE',
 'ESTCIVMAE',
 'ESCMAE',
 'QTDFILVIVO',
 'GESTACAO',
 'GRAVIDEZ',
 'CONSULTAS',
 'APGAR5',
 'Classificação']

In [157]:
colunas2 = ['loc_nasc', 'idade_mae', 'est_civ_mae', 'esc_mae', 'qtd_fil_vivo', 'gestacao', 
            'gravidez', 'consultas', 'apgar_5', 'classificacao']

sinasc_tratado.columns = colunas2
sinasc_tratado

,loc_nasc,idade_mae,est_civ_mae,esc_mae,qtd_fil_vivo,gestacao,gravidez,consultas,apgar_5,classificacao
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,normal
